<a href="https://colab.research.google.com/github/damilojohn/Playlist_Generator/blob/main/Generating_Afrobeats_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing necessary libraries
!pip install sentence-transformers
!pip install datasets 
!pip install wandb

In [ ]:
#text file containing lyrics
!wget https://raw.githubusercontent.com/damilojohn/Playlist_Generator/main/model_embedding.txt
#songs csv 
!wget https://raw.githubusercontent.com/damilojohn/Playlist_Generator/main/Pandas%20transformation/spotify_transformed.csv

In [ ]:
#setting up wandb
!wandb login
import wandb

In [ ]:
config = {
    'Name' : 'Embeddings for afrobeats lyrics',
    'Dataset' : 'spotify api lyrics',
    'Model' : 'msmarco-miniLM-pretrained',
    'notes':'No text cleaning was performed yet.creating a baseline performance'
}
wandb.init(project='Playlist Generator',entity='damilojohn',config=config)

In [3]:
from sentence_transformers import SentenceTransformer,util
from tqdm.auto import tqdm
import pandas as pd 
import transformers
import pickle 
from huggingface_hub import hf_hub_download
from datasets import load_dataset
import re

In [ ]:
#loading song lyrics into a dataset object 
data_path = '/content/model_embedding.txt'
songs = load_dataset('text',
                     data_files={'train':[data_path]})

In [ ]:
symmetric_embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
asymmetric_embedder = SentenceTransformer('msmarco-distilbert-base-v4')

In [6]:
songs= pd.read_csv('/content/spotify_transformed.csv')

In [ ]:
songs.drop('Unnamed: 0',axis=1,inplace=True)
songs.head(5)

In [19]:
verses = open('/content/model_embedding.txt','r')
lyrics = verses.read()
verses.close()

In [21]:
lyrics = lyrics.split('\\n')

In [ ]:
#removing empty characters our list 
test2 = [i for i in lyrics if i]
#removing square brackets and other jargons 
for i,lyrics in enumerate(test2) :
  if not lyrics[0].isalpha():
    del test2[i]
test2

In [37]:
#splitting by verses, just kidding , just kidding 
' '.join(lyrics).split('[')[5]

"Verse 2: Wizkid] Say na me dey mess up your mind And na me dey make you free up your mind Say your body talk to me nice Say na my love, you dey need for your life Yeah our love no be lie See yeah we dey together yeah day and night Yeah if I leave you go kpai Yeah if you leave I go kpai (Oh, oh woah) I'm stroking your body, baby, loving your body, baby As you're whinin' your body, baby, so crazy Lovin' your body, baby, gyal iwo ni temi I just wanna let you know, oh, woah, hope every day that I dey by you  You might also like "

In [41]:
import copy 
lyrics_cleaned = copy.deepcopy(test2)

In [ ]:
lyrics_cleaned

In [ ]:
#trying out regex later 
pattern = re.compile('\[')


In [49]:
embeddings = asymmetric_embedder.encode(lyrics_cleaned,)
assert len(lyrics_cleaned) == len(embeddings) , 'wahala dey'

In [55]:
#trying out semantic search 
query = ['I just want to cruise and jolly']
query_embedding = asymmetric_embedder.encode(query)
hits = util.semantic_search(query_embedding,embeddings,top_k = 6)
hits

[[{'corpus_id': 3291, 'score': 0.39620208740234375},
  {'corpus_id': 3550, 'score': 0.3827310800552368},
  {'corpus_id': 3345, 'score': 0.3679012656211853},
  {'corpus_id': 2796, 'score': 0.3589097559452057},
  {'corpus_id': 1711, 'score': 0.3511357605457306},
  {'corpus_id': 1734, 'score': 0.3511357605457306}]]

In [56]:
lyrics_cleaned[3291]

'I dey catch cruise, Eko Miami'